# MCP

In [35]:
# Custom Model Context Provider (MCP) Server Implementation
# This is a custom abstraction layer for interacting with various LLM APIs
# Not to be confused with any official standard - this is a custom implementation

# Import necessary libraries
import os
import json
import time
import logging
import requests
from typing import Dict, List, Any, Optional, Union
import openai
from functools import wraps
try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    print("dotenv not installed. Using environment variables directly.")

In [ ]:
# Configuration
config = {
    "api": {
        "openai": {
            "api_key": os.getenv("OPENAI_API_KEY", ""),  # get frrom .env file or environment variable
            "default_model": "gpt-4o",
            "temperature": 0.7,
            "max_tokens": 1000
        },
        # Placeholder for other APIs that can be added later
        "anthropic": {
            "api_key": os.getenv("ANTHROPIC_API_KEY", ""),
            "enabled": False,
            "default_model": "claude-3-opus-20240229",
            "temperature": 0.7,
            "max_tokens": 1000
        },
        "gemini": {
            "api_key": os.getenv("GEMINI_API_KEY", ""),
            "enabled": False
        }
    },
    "server": {
        "log_level": "INFO",
        "timeout": 30,
        "retry_attempts": 3,
        "retry_delay": 2,  # seconds between retry attempts
        "cache_enabled": True,
        "cache_ttl": 3600  # in seconds
    }
}

# Setup logging based on configuration
logging.basicConfig(
    level=getattr(logging, config['server']['log_level']),
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger('MCP')

In [37]:
# API Client Classes

# Retry decorator for API calls
def retry_on_exception(max_attempts, delay):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            attempts = 0
            last_exception = None
            
            while attempts < max_attempts:
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    attempts += 1
                    last_exception = e
                    logger.warning(f"Attempt {attempts} failed with error: {e}. Retrying in {delay} seconds...")
                    if attempts < max_attempts:
                        time.sleep(delay)
            
            logger.error(f"All {max_attempts} attempts failed. Last error: {last_exception}")
            raise last_exception
        return wrapper
    return decorator

class BaseAPIClient:
    """Base class for API clients"""
    
    def __init__(self, api_config: Dict[str, Any], server_config: Dict[str, Any] = None):
        self.config = api_config
        self.server_config = server_config or {"retry_attempts": 3, "retry_delay": 2, "timeout": 30}
        self.name = "base"
        self.logger = logging.getLogger(f"MCP.{self.name}")
    
    def validate_config(self) -> bool:
        """Validate if the configuration is sufficient to use this API"""
        raise NotImplementedError("Subclasses must implement this method")
    
    def generate_text(self, prompt: str, options: Optional[Dict[str, Any]] = None) -> str:
        """Generate text based on a prompt"""
        raise NotImplementedError("Subclasses must implement this method")


class OpenAIClient(BaseAPIClient):
    """Client for OpenAI API"""
    
    def __init__(self, api_config: Dict[str, Any], server_config: Dict[str, Any] = None):
        super().__init__(api_config, server_config)
        self.name = "openai"
        openai.api_key = api_config.get("api_key", "")
    
    def validate_config(self) -> bool:
        """Check if OpenAI API key is available"""
        valid = bool(self.config.get("api_key", ""))
        if not valid:
            self.logger.error("OpenAI API key is missing")
        return valid
    
    @retry_on_exception(max_attempts=3, delay=2)
    def generate_text(self, prompt: str, options: Optional[Dict[str, Any]] = None) -> str:
        """Generate text using OpenAI API"""
        if not self.validate_config():
            raise ValueError("OpenAI API key is not configured")
        
        opts = options or {}
        model = opts.get("model", self.config.get("default_model", "gpt-4o"))
        temperature = opts.get("temperature", self.config.get("temperature", 0.7))
        max_tokens = opts.get("max_tokens", self.config.get("max_tokens", 1000))
        
        self.logger.info(f"Generating text with OpenAI model: {model}")
        try:
            response = openai.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": prompt}
                ],
                temperature=temperature,
                max_tokens=max_tokens
            )
            return response.choices[0].message.content
        except Exception as e:
            self.logger.error(f"Error generating text with OpenAI: {e}")
            raise

class AnthropicClient(BaseAPIClient):
    """Client for Anthropic Claude API"""
    
    def __init__(self, api_config: Dict[str, Any], server_config: Dict[str, Any] = None):
        super().__init__(api_config, server_config)
        self.name = "anthropic"
        self.api_key = api_config.get("api_key", "")
    
    def validate_config(self) -> bool:
        """Check if Anthropic API key is available"""
        valid = bool(self.api_key)
        if not valid:
            self.logger.error("Anthropic API key is missing")
        return valid
    
    @retry_on_exception(max_attempts=3, delay=2)
    def generate_text(self, prompt: str, options: Optional[Dict[str, Any]] = None) -> str:
        """Generate text using Anthropic API"""
        if not self.validate_config():
            raise ValueError("Anthropic API key is not configured")
            
        opts = options or {}
        model = opts.get("model", self.config.get("default_model", "claude-3-opus-20240229"))
        temperature = opts.get("temperature", self.config.get("temperature", 0.7))
        max_tokens = opts.get("max_tokens", self.config.get("max_tokens", 1000))
        
        self.logger.info(f"Generating text with Anthropic model: {model}")
        try:
            headers = {
                "x-api-key": self.api_key,
                "content-type": "application/json"
            }
            
            payload = {
                "model": model,
                "messages": [
                    {"role": "user", "content": prompt}
                ],
                "temperature": temperature,
                "max_tokens": max_tokens
            }
            
            response = requests.post(
                "https://api.anthropic.com/v1/messages",
                headers=headers,
                json=payload,
                timeout=self.server_config.get("timeout", 30)
            )
            
            response.raise_for_status()
            return response.json()["content"][0]["text"]
        except Exception as e:
            self.logger.error(f"Error generating text with Anthropic: {e}")
            raise


class APIClientFactory:
    """Factory class to create the appropriate API client"""
    
    @staticmethod
    def create_client(api_name: str, config: Dict[str, Any]) -> BaseAPIClient:
        """Create an API client based on the name"""
        server_config = config.get("server", {})
        
        if api_name == "openai":
            return OpenAIClient(config["api"]["openai"], server_config)
        elif api_name == "anthropic" and config["api"]["anthropic"].get("enabled", False):
            return AnthropicClient(config["api"]["anthropic"], server_config)
        # Add more clients here as they become available
        else:
            logger.error(f"Unsupported API client: {api_name}")
            raise ValueError(f"Unsupported API client: {api_name}")

In [38]:
# MCP Classes

class Model:
    """Model class that handles interaction with the API clients"""
    
    def __init__(self, api_client: BaseAPIClient):
        self.api_client = api_client
        self.logger = logging.getLogger(f"MCP.Model.{api_client.name}")
    
    def generate(self, prompt: str, options: Optional[Dict[str, Any]] = None) -> str:
        """Generate a response using the API client"""
        self.logger.debug(f"Generating response for prompt: {prompt[:50]}...")
        try:
            return self.api_client.generate_text(prompt, options)
        except Exception as e:
            self.logger.error(f"Failed to generate response: {e}")
            return f"Error: Failed to generate response. {str(e)}"


class Controller:
    """Controller class that manages the flow of requests and responses"""
    
    def __init__(self, model: Model, config: Dict[str, Any]):
        self.model = model
        self.config = config
        self.cache = {} if config["server"].get("cache_enabled", False) else None
        self.logger = logging.getLogger("MCP.Controller")
    
    def process_request(self, request_data: Dict[str, Any]) -> Dict[str, Any]:
        """Process a request and return a response"""
        try:
            prompt = request_data.get("prompt", "")
            options = request_data.get("options", {})
            
            if not prompt:
                self.logger.warning("Received empty prompt")
                return {
                    "response": "Error: Empty prompt",
                    "error": True,
                    "timestamp": time.time()
                }
            
            self.logger.info(f"Processing request with prompt: {prompt[:50]}...")
            
            # Check cache if enabled
            cache_key = f"{prompt}:{json.dumps(options)}"
            if self.cache is not None and cache_key in self.cache:
                cache_entry = self.cache[cache_key]
                if time.time() - cache_entry["timestamp"] < self.config["server"].get("cache_ttl", 3600):
                    self.logger.info("Returning cached response")
                    return {
                        "response": cache_entry["response"],
                        "cached": True,
                        "timestamp": time.time()
                    }
            
            # Generate response
            response = self.model.generate(prompt, options)
            
            # Update cache if enabled
            if self.cache is not None:
                self.cache[cache_key] = {
                    "response": response,
                    "timestamp": time.time()
                }
            
            return {
                "response": response,
                "cached": False,
                "timestamp": time.time()
            }
        except Exception as e:
            self.logger.error(f"Error processing request: {e}")
            return {
                "response": f"Error: {str(e)}",
                "error": True,
                "timestamp": time.time()
            }


class Pipeline:
    """Pipeline class that orchestrates the entire process"""
    
    def __init__(self, config: Dict[str, Any]):
        self.config = config
        self.controller = None
        self.logger = logging.getLogger("MCP.Pipeline")
        self.initialize()
    
    def initialize(self):
        """Initialize the pipeline with the configured API"""
        try:
            # Check enabled APIs and select the first available one
            api_priority = ["openai", "anthropic", "gemini"]
            selected_api = None
            
            for api_name in api_priority:
                api_config = self.config["api"].get(api_name, {})
                if api_name == "openai" or api_config.get("enabled", False):
                    if api_config.get("api_key"):
                        selected_api = api_name
                        break
            
            if selected_api is None:
                raise ValueError("No valid API configuration found")
                
            self.logger.info(f"Initializing pipeline with {selected_api} API")
            api_client = APIClientFactory.create_client(selected_api, self.config)
            model = Model(api_client)
            self.controller = Controller(model, self.config)
        except Exception as e:
            self.logger.error(f"Failed to initialize pipeline: {e}")
            raise
    
    def process(self, request_data: Dict[str, Any]) -> Dict[str, Any]:
        """Process a request through the pipeline"""
        if self.controller is None:
            self.logger.error("Pipeline has not been initialized")
            raise ValueError("Pipeline has not been initialized")
        
        return self.controller.process_request(request_data)

In [39]:
# Helper Functions

def load_config_from_file(file_path: str) -> Dict[str, Any]:
    """Load configuration from a JSON file"""
    try:
        with open(file_path, 'r') as f:
            return json.load(f)
    except Exception as e:
        logger.error(f"Error loading config from {file_path}: {e}")
        return {}

def merge_config(base_config: Dict[str, Any], override_config: Dict[str, Any]) -> Dict[str, Any]:
    """Merge override configuration into base configuration"""
    merged = base_config.copy()
    
    for key, value in override_config.items():
        if key in merged and isinstance(merged[key], dict) and isinstance(value, dict):
            merged[key] = merge_config(merged[key], value)
        else:
            merged[key] = value
    
    return merged

In [40]:
# Example: How to add a new API Client

# This is an example implementation of another API client (e.g., for Gemini)
'''
class GeminiClient(BaseAPIClient):
    """Client for Google Gemini API"""
    
    def __init__(self, api_config: Dict[str, Any], server_config: Dict[str, Any] = None):
        super().__init__(api_config, server_config)
        self.name = "gemini"
        self.api_key = api_config.get("api_key", "")
    
    def validate_config(self) -> bool:
        """Check if Gemini API key is available"""
        valid = bool(self.api_key)
        if not valid:
            self.logger.error("Gemini API key is missing")
        return valid
    
    @retry_on_exception(max_attempts=3, delay=2)
    def generate_text(self, prompt: str, options: Optional[Dict[str, Any]] = None) -> str:
        """Generate text using Gemini API"""
        if not self.validate_config():
            raise ValueError("Gemini API key is not configured")
        
        # Implement Gemini API call here
        # ...
        return "Gemini response"

# To update the factory, add:
# elif api_name == "gemini" and config["api"]["gemini"].get("enabled", False):
#     return GeminiClient(config["api"]["gemini"], server_config)
'''

'\nclass GeminiClient(BaseAPIClient):\n    """Client for Google Gemini API"""\n    \n    def __init__(self, api_config: Dict[str, Any], server_config: Dict[str, Any] = None):\n        super().__init__(api_config, server_config)\n        self.name = "gemini"\n        self.api_key = api_config.get("api_key", "")\n    \n    def validate_config(self) -> bool:\n        """Check if Gemini API key is available"""\n        valid = bool(self.api_key)\n        if not valid:\n            self.logger.error("Gemini API key is missing")\n        return valid\n    \n    @retry_on_exception(max_attempts=3, delay=2)\n    def generate_text(self, prompt: str, options: Optional[Dict[str, Any]] = None) -> str:\n        """Generate text using Gemini API"""\n        if not self.validate_config():\n            raise ValueError("Gemini API key is not configured")\n        \n        # Implement Gemini API call here\n        # ...\n        return "Gemini response"\n\n# To update the factory, add:\n# elif api_nam

In [41]:
# Main Execution

def main():
    try:
        # Initialize the pipeline with our config
        logger.info("Initializing MCP pipeline")
        pipeline = Pipeline(config)
        
        # Example requests
        example_requests = [
            {
                "prompt": "Explain what an MCP server is in simple terms",
                "options": {
                    "temperature": 0.5,
                    "max_tokens": 200
                }
            },
            {
                "prompt": "Write a haiku about artificial intelligence",
                "options": {
                    "temperature": 0.9,
                    "max_tokens": 50
                }
            }
        ]
        
        # Process each request and print results
        for i, request in enumerate(example_requests):
            logger.info(f"Processing request {i+1}")
            print(f"\n--- Request {i+1} ---")
            print(f"Prompt: {request['prompt']}")
            
            response_data = pipeline.process(request)
            
            print("\n--- Response ---")
            print(f"Cached: {response_data.get('cached', False)}")
            print(f"Error: {response_data.get('error', False)}")
            print(f"Response:\n{response_data['response']}")
            print("-" * 50)
    except Exception as e:
        logger.error(f"Main execution failed: {e}")
        print(f"Error: {e}")

if __name__ == "__main__":
    main()

2025-04-06 11:58:22,873 - MCP - INFO - Initializing MCP pipeline
2025-04-06 11:58:22,874 - MCP.Pipeline - INFO - Initializing pipeline with openai API
2025-04-06 11:58:22,874 - MCP - INFO - Processing request 1
2025-04-06 11:58:22,875 - MCP.Controller - INFO - Processing request with prompt: Explain what an MCP server is in simple terms...
2025-04-06 11:58:22,876 - MCP.base - INFO - Generating text with OpenAI model: gpt-4o



--- Request 1 ---
Prompt: Explain what an MCP server is in simple terms


2025-04-06 11:58:26,431 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-06 11:58:26,435 - MCP - INFO - Processing request 2
2025-04-06 11:58:26,436 - MCP.Controller - INFO - Processing request with prompt: Write a haiku about artificial intelligence...
2025-04-06 11:58:26,436 - MCP.base - INFO - Generating text with OpenAI model: gpt-4o



--- Response ---
Cached: False
Error: False
Response:
An MCP server typically refers to a "Minecraft Server" or "Multi-Channel Processor" server, depending on the context. However, in most casual discussions, an MCP server usually means a Minecraft Server.

In simple terms, a Minecraft Server is a computer or a program that allows multiple people to connect and play Minecraft together over the internet or a local network. It hosts the game world, manages player connections, and ensures that everyone sees the same game state. Players can join the server to explore, build, and interact in the shared Minecraft environment.

If you're referring to a Multi-Channel Processor server, it would be related to a system that manages multiple data streams or channels, often used in telecommunications or broadcasting. However, this is less common in casual discussions.

If you have a specific context in mind, let me know, and I can provide a more precise explanation!
-------------------------------

2025-04-06 11:58:27,386 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



--- Response ---
Cached: False
Error: False
Response:
Circuits hum with thought,  
Silicon dreams awaken—  
Mind of code and light.
--------------------------------------------------


In [42]:
# Run the main function
main()

2025-04-06 11:58:27,401 - MCP - INFO - Initializing MCP pipeline
2025-04-06 11:58:27,402 - MCP.Pipeline - INFO - Initializing pipeline with openai API
2025-04-06 11:58:27,402 - MCP - INFO - Processing request 1
2025-04-06 11:58:27,403 - MCP.Controller - INFO - Processing request with prompt: Explain what an MCP server is in simple terms...
2025-04-06 11:58:27,404 - MCP.base - INFO - Generating text with OpenAI model: gpt-4o



--- Request 1 ---
Prompt: Explain what an MCP server is in simple terms


2025-04-06 11:58:29,857 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-06 11:58:29,869 - MCP - INFO - Processing request 2
2025-04-06 11:58:29,870 - MCP.Controller - INFO - Processing request with prompt: Write a haiku about artificial intelligence...
2025-04-06 11:58:29,870 - MCP.base - INFO - Generating text with OpenAI model: gpt-4o



--- Response ---
Cached: False
Error: False
Response:
An MCP server typically refers to a server that runs the "MCP" software or protocol. In the context of computing, MCP can stand for different things depending on the specific technology or company. 

One common reference is to the Master Control Program (MCP), which is the operating system used by Unisys ClearPath mainframe computers. In this context, an MCP server would be a server running the MCP operating system, which is designed to manage and execute tasks on a mainframe computer, providing a stable and secure environment for large-scale computing needs.

Another context where MCP might be used is in gaming or other software systems, but the specific meaning can vary. If you have a specific context or application in mind, providing more details would help clarify the exact type of MCP server you're referring to.
--------------------------------------------------

--- Request 2 ---
Prompt: Write a haiku about artificial intelli

2025-04-06 11:58:30,567 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



--- Response ---
Cached: False
Error: False
Response:
Machines learn and grow,  
Whispers of code spark new thoughts—  
Metal minds, alive.
--------------------------------------------------
